In [2]:
import random, math
import numpy as np
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, Dataset

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision.datasets import ImageFolder

import sys, shutil
from os import listdir, mkdir
from PIL import Image
from collections import Counter, defaultdict

from IPython.display import display, clear_output

In [3]:
ORIG_IMAGE_PATH = '../data/images'
ORIG_LABEL_PATH = '../data/annotations'
LABEL_PATH = '../data/annotations2'
IMAGE_PATH = '../data/images2'
RANDOM_SEED = 1234

In [4]:
def get_class_map():
    classnametoint = {}
    inttoclassname = {}

    i = 0
    label_files = sorted(listdir(ORIG_LABEL_PATH))
    for fname in label_files:
        img_class, _ = fname.split('.')
        classnametoint[img_class] = i
        inttoclassname[i] = img_class
        i += 1

    return classnametoint, inttoclassname

In [5]:
def train_valid_test_split(train_fr=.6, valid_fr=.2, test_fr=.2):
    """
    Creates subfolders train, test, validation. 
    Splits annotation files into train, validation and test sets.
    """
    random.seed(RANDOM_SEED)
    
    for dir_name in ['', '/train', '/validation', '/test']: 
        try:
            mkdir(f'{LABEL_PATH}{dir_name}')
        except:
            pass
      
    # Make the split
    for fname in listdir(ORIG_LABEL_PATH):
        with open(f'{ORIG_LABEL_PATH}/{fname}', 'r') as fh:
            img_ids = fh.read().splitlines()
            random.shuffle(img_ids)
            split1, split2 = math.ceil(test_fr * len(img_ids)), math.ceil(valid_fr * len(img_ids))
            test_ids, valid_ids, train_ids = img_ids[:split1], img_ids[split1:split1+split2], img_ids[split1+split2:]
            with open(f'{LABEL_PATH}/test/{fname}', 'w') as outfile:
                outfile.write('\n'.join(test_ids))
            with open(f'{LABEL_PATH}/validation/{fname}', 'w') as outfile:
                outfile.write('\n'.join(valid_ids))
            with open(f'{LABEL_PATH}/train/{fname}', 'w') as outfile:
                outfile.writelines('\n'.join(train_ids))
            print(f'Wrote {len(train_ids)}, {len(valid_ids)}, {len(test_ids)} lines (of {len(img_ids)}) in {fname} (train, validation, test)')

# Need to run only once. 
if False:
    train_valid_test_split()

In [6]:
def copy_images_to_subfolders():
    """
    Copy images to subfolders by set (train, validation, test)
    """
    log = []
    # Loop through each set
    for setname in listdir(LABEL_PATH):
        display(f'Processing set: {setname}')
        try:
            mkdir(f'{IMAGE_PATH}/{setname}')
            mkdir(f'{IMAGE_PATH}/{setname}/0')  # Just an arbitrary subfolder to support using ImageFolder dataset
        except:
            pass
        # Loop through each label
        for fname in listdir(f'{LABEL_PATH}/{setname}'):
            with open(f'{LABEL_PATH}/{setname}/{fname}', 'r') as fh:
                img_class, _ = fname.split('.')
                clear_output(wait=True)
                display(f'Processing class: {img_class}')
                destdir = f'{IMAGE_PATH}/{setname}/0'
                # Loop through each image
                for idx in fh.read().splitlines():
                    shutil.copy(f'{ORIG_IMAGE_PATH}/im{idx}.jpg', destdir)
                log.append(f'Copied {len(listdir(destdir))} files in {destdir}')
        
    print('\n'.join(log))
    print("\nDone")


# No need to run multiple times
if False:
    try:
        mkdir(f'{IMAGE_PATH}')
    except:
        pass
    copy_images_to_subfolders()

In [7]:
def compute_batch_statistics(bs, max_items, root_path):
    """
    We need to run this only once with given dataset
    From https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7?u=laochanlam
    """
    print(f'\nProcessing folder: {root_path}')
    
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    dataset = ImageFolder(root=root_path, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=4)

    pop_mean = []
    pop_std = []
    for i, data in enumerate(dataloader):
        numpy_image = data[0].numpy().reshape(len(data[0]), 3, 128, 128)
        pop_mean.append(np.mean(numpy_image, axis=(0,2,3)))
        pop_std.append(np.std(numpy_image, axis=(0,2,3), ddof=1)) # ddof=1  => Sample standard deviation

    mean, std = np.array(pop_mean).mean(axis=0), np.array(pop_std).mean(axis=0)
    
    print(f'Batch mean {mean}, std {std}')
    return mean, std

# Default values (calculated from train and validation data)
MEAN_tr = (0.4359158, 0.4008995, 0.36738247)
STD_tr = (0.30338776, 0.28947875, 0.29492074)
MEAN_val = (0.4338689, 0.40064156, 0.3681074)
STD_val = (0.3044515, 0.29112726, 0.2954069)
MEAN_test = (0.43964133, 0.40063056, 0.3668358)
STD_test = (0.3038753, 0.29096046, 0.29495955)

# No need to run multiple times
if False:
    MEAN_tr, STD_tr = compute_batch_statistics(bs=64, max_items=256, root_path=f'{IMAGE_PATH}/train/')
    MEAN_val, STD_val = compute_batch_statistics(bs=64, max_items=256, root_path=f'{IMAGE_PATH}/validation/')
    MEAN_test, STD_test = compute_batch_statistics(bs=64, max_items=256, root_path=f'{IMAGE_PATH}/test/')

In [16]:
class TheThreeCaballerosDataset(Dataset):
    def __init__(self, labels_root, images_root, transformations=None):
        self.skipped = []
        self.labels_root = labels_root
        self.images_root = images_root
        self.labels, self.images = self.get_data()
        self.keys = list(self.labels.keys())
        self.transformations = transformations
        
    def get_data(self):
        images = {}
        label_files = listdir(self.labels_root)
        labels = defaultdict(lambda: [False]*len(label_files))    # Init to array [False, False, ...]
        # Loop through labels
        for idx, fname in enumerate(sorted(label_files)):
            img_class, _ = fname.split('.')
            class_int = class_map[img_class]
            # open the annotation file
            with open(f'{self.labels_root}{fname}', 'r') as fh: 
                # get image ids from annotation file
                img_ids = fh.read().splitlines()
                # extract image data
                for im_id in sorted(img_ids):
                    key = int(im_id)
                    # image
                    if not im_id in images:
                        img = self.get_image_data(f'{self.images_root}/im{im_id}.jpg')
                        if img is None:
                            self.skipped.append(im_id)
                            continue
                    images[key] = img
                    # label
                    labels[key][class_int] = True
                    
        return labels, images
    
    def get_image_data(self, image_path):
        img = Image.open(image_path)
        img_data = np.asarray(img)#.convert("RGB"))
        img.close()
        if len(img_data.shape) == 3:
            return img_data.flatten().astype(np.float32)
        return None
        
    def __getitem__(self, idx):
        image_id = self.keys[idx]
        image = self.images[image_id]
        # Apply transformations
        try:
            if (self.transformations):
                image = self.transformations(image)
            # return tuple
            return (image, self.labels[image_id])
        except Exception as e:
            print(f'Image index={idx}, image_id={image_id}, shape={image.shape}')
            raise e

    def __len__(self):
        return len(self.labels)

In [9]:
def get_dataloader(labels_root, images_root, bs, transformations):

    dataset = TheThreeCaballerosDataset(labels_root, images_root, transformations)
    dataloader = DataLoader(dataset, batch_size=bs, shuffle=True)
    
    return dataloader

In [10]:
class TwoLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_classes):
        super().__init__()

        self.input_layer = nn.Linear(n_input, n_hidden1)
        self.hidden1 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden2 = nn.Linear(n_hidden2, n_classes)
        self.relu = nn.ReLU()
        self.bn0 = nn.BatchNorm1d(n_input)
        self.bn1 = nn.BatchNorm1d(n_hidden1)
        self.bn2 = nn.BatchNorm1d(n_hidden2)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.hidden2(x)

        return x

In [11]:
class OneLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden, n_classes):
        super().__init__()
        
        self.bn0 = nn.BatchNorm1d(n_input)
        self.input_layer = nn.Linear(n_input, n_hidden)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(n_hidden)
        self.hidden = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        x = self.bn0(x)
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.hidden(x)

        return x

In [12]:
def train(dataloader, model, optimizer, criterion, device, n_epochs=50, losses=[]):

    log = []
    model.train()

    for epoch in range(n_epochs):
        
        for i, batch in enumerate(dataloader):
            X, y = batch
            X = X.reshape(X.shape[0], 128*128*3)
            
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            
            losses.append(loss)
            
            clear_output(wait=True)
            display(f'Epoch: {epoch+1}, iteration: {i+1}, loss: {loss}')

        log.append(f'Epoch: {epoch+1}, loss: {loss}')
    
    print('\n'.join(log))
    print('\nDone training')

In [13]:
use_cuda = False

device = torch.device('cuda') if use_cuda else torch.device('cpu')

lr = 0.01
n_epochs = 2 #10
bs = 64 #256

class_map, _ = get_class_map()
n_classes = len(class_map.keys())
    
#model = TwoLayerModel(128*128*3, 128, 64, n_classes).to(device)
model = OneLayerModel(128*128*3, 128, n_classes).to(device)   # (self, D_in, H, D_out):

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [15]:
transformations = {
    'train' : transforms.Compose([
            #transforms.ToPILImage(),
            #transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN_tr, std=STD_tr)
        ]),
    'validation': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN_val, std=STD_val)
        ]),
    'test': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN_test, std=STD_test)
        ])
}

#train_dataloader = get_dataloader(
#    labels_root=f'{LABEL_PATH}/train/', images_root=f'{IMAGE_PATH}/train/0/', bs=bs, transformations=transformations['train'])

#print(train_dataloader.dataset.skipped)

validation_dataloader = get_dataloader(
    labels_root=f'{LABEL_PATH}/validation/', images_root=f'{IMAGE_PATH}/validation/0/', bs=bs, transformations=transformations['validation'])

print(validation_dataloader.dataset.skipped)

AttributeError: 'TheThreeCaballerosDataset' object has no attribute 'skipped'

In [18]:
(128*128, 128*128*3)

(16384, 49152)

In [46]:
#train(train_dataloader, model, optimizer, criterion, device, n_epochs)
train(validation_dataloader, model, optimizer, criterion, device, n_epochs)

Image index=2022, image_id=11388, shape=(49152,)


ValueError: pic should be 2/3 dimensional. Got 1 dimensions.